# Feature Extraction using Convolutional Neural Network

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import InputLayer,Conv2D,BatchNormalization,MaxPool2D,Dropout,Flatten,LSTM,Dense,Reshape
from tensorflow.keras import Sequential
import numpy as np
import csv

In [8]:
feature_extraction_layers = [
    InputLayer(input_shape=(15,1025,151),name="Input Layer"),
    Conv2D(filters=64,kernel_size=(5,5),activation="relu",name="Convolution_Layer_1"),
    MaxPool2D(pool_size=(4,4)),
    BatchNormalization(),
    Dropout(rate=0.4),
    Conv2D(filters=128,kernel_size=(2,2),activation="relu",name="Convolution_Layer_2"),
    MaxPool2D(pool_size=(1,1)),
    BatchNormalization(),
    Reshape((254,128)),
    LSTM(128),
    Dropout(0.4),
    Dense(60,activation="relu"),
    Dense(2)
    
]

In [9]:
feature_extraction_model = Sequential(feature_extraction_layers)

In [31]:
feature_extraction_model.compile(optimizer=tf.keras.optimizers.Adam(0.001,decay=0)
                                 ,metrics=["accuracy"] ,loss=tf.keras.losses.binary_crossentropy)

In [32]:
feature_extraction_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Convolution_Layer_1 (Conv2D) (None, 11, 1021, 64)      241664    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 2, 255, 64)        0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 2, 255, 64)        256       
_________________________________________________________________
dropout_4 (Dropout)          (None, 2, 255, 64)        0         
_________________________________________________________________
Convolution_Layer_2 (Conv2D) (None, 1, 254, 128)       32896     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 1, 254, 128)       0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 1, 254, 128)      

In [63]:
def create_dataset():
    features = []
    labels=[]
    training_folder="./training/"
    with open("./training/dataset.csv","r") as csv_file:
        file = csv.reader(csv_file)
        for row in file:
            features.append(np.reshape(np.load(training_folder+row[0]),(15,1025,151)))
            labels.append(np.array([int(row[1]),int(row[2])]))
    features = np.array(features)
    labels = np.array(labels)
    dataset = tf.data.Dataset.from_tensor_slices((features,labels))
    dataset = dataset.as_numpy_iterator()
    return features,labels

    

In [ ]:
features,labels = create_dataset()
labels = tf.keras.utils.to_categorical(labels)
feature_extraction_model.fit(features,labels,epochs=30)
# data = np.reshape(data,(1,15,1025,151))
# prediction = feature_extraction_model.fit(data)
# prediction.shape